# All Pairs Shortest Path
_All Pairs Shortest Path_ calculates the shortest weight path between all pairs of nodes.
An algorithm to solve this is Floyd Warshall or Parallel Johnson's algorithm.


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

In [3]:
create_graph_query = '''
CREATE (a:Loc{name:'A'}), (b:Loc{name:'B'}), (c:Loc{name:'C'}), 
       (d:Loc{name:'D'}), (e:Loc{name:'E'}), (f:Loc{name:'F'}),
       (a)-[:ROAD {cost:50}]->(b),
       (a)-[:ROAD {cost:50}]->(c),
       (a)-[:ROAD {cost:100}]->(d),
       (a)-[:RAIL {cost:50}]->(d),
       (b)-[:ROAD {cost:40}]->(d),
       (c)-[:ROAD {cost:40}]->(d),
       (c)-[:ROAD {cost:80}]->(e),
       (d)-[:ROAD {cost:30}]->(e),
       (d)-[:ROAD {cost:80}]->(f),
       (e)-[:ROAD {cost:40}]->(f),
       (e)-[:RAIL {cost:20}]->(f);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 6, 'relationships-created': 11, 'nodes-created': 6, 'properties-set': 17}


In [4]:
streaming_query = """
CALL algo.allShortestPaths.stream('cost',{nodeQuery:'Loc',defaultValue:1.0})
YIELD sourceNodeId, targetNodeId, distance
WITH sourceNodeId, targetNodeId, distance 
WHERE algo.isFinite(distance) = true
RETURN sourceNodeId, targetNodeId, distance LIMIT 20
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,sourceNodeId,targetNodeId,distance
0,134,134,0.0
1,16,16,0.0
2,177,134,110.0
3,177,177,0.0
4,177,178,40.0
5,177,179,70.0
6,16,134,110.0
7,16,178,40.0
8,16,179,70.0
9,15,15,0.0
